In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
import time
import statsmodels.api as sm
from sklearn import linear_model
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.preprocessing import StandardScaler

import os

from sklearn.model_selection import train_test_split

In [5]:
path = 'data'+os.path.sep
df = pd.read_csv(path+'market_info_soul.csv',encoding='utf-8',delimiter='|')

In [6]:
df = df[['상호명','상가업소번호','지점명','상권업종대분류명','상권업종중분류명','상권업종소분류명','표준산업분류명','시도명','법정동명','경도','위도']]

In [7]:
df[df['상권업종대분류명']=='학문/교육']['상권업종중분류명'].unique()

array(['학원-어학', '학원-음악미술무용', '학원-보습교습입시', '학원-예능취미체육', '유아교육', '학원기타',
       '학문교육기타', '학원-자격/국가고시', '학원-컴퓨터', '도서관/독서실', '학원-창업취업취미'],
      dtype=object)

In [52]:
df[df['상권업종대분류명']=='학문/교육'].groupby(['상권업종중분류명','상권업종소분류명']).count()

상호명  상가업소번호  지점명  상권업종대분류명  표준산업분류명   시도명  행정동명  \
상권업종중분류명   상권업종소분류명                                                        
도서관/독서실    고시원           853     853   47       853      853   853   853   
           독서실           914     914   56       914      914   914   914   
           이동도서관운영        18      18    1        18       18    18    18   
유아교육       놀이방           116     116    1       116      116   116   116   
           동화구연            2       2    0         2        2     2     2   
           블럭놀이방          17      17    1        17       17    17    17   
           실내놀이터-유아       71      71   11        71       71    71    71   
           어린이집         3700    3700    7      3700     3700  3700  3700   
           유아교육기관기타       90      90    7        90       90    90    90   
학문교육기타     시험정보안내          5       5    0         5        5     5     5   
           운동/코치학교        53      53    4        53       53    53    53   
           학습지보급         424     424   77       424      424   424   424   
학원-보습교습입시  학원-검정고시         7       7    3         7        7     7     7   
           학원-입시        6706    6706  656      6706     6706  6706  6706   
           학원-편입          24      24   10        24       24    24    24   
학원-어학      학원-어린이영어      306     306   81       306      306   306   306   
           학원-외국어/어학    3269    3269  430      3269     3269  3269  3268   
학원-예능취미체육  검도도장           97      97    1        97       97    97    97   
           레크레이션지도/상담      3       3    0         3        3     3     3   
           유도장            46      46    2        46       46    46    46   
           태권도장          640     640    5       640      640   640   640   
           학원-가요          35      35    3        35       35    35    35   
           학원-기억법          4       4    0         4        4     4     4   
           학원-꽃꽃이         25      25    5        25       25    25    25   
           학원-레크리에이션       1       1    0         1        1     1     1   
           학원-마술           6       6    0         6        6     6     6   
           학원-바둑          34      34    2        34       34    34    34   
           학원-배우/탤런트      84      84   18        84       84    84    84   
           학원-사교춤         46      46   11        46       46    46    46   
           학원-사진           2       2    1         2        2     2     2   
           학원-서당          18      18    2        18       18    18    18   
           학원-스포츠/재즈댄스   244     244   23       244      244   244   244   
           학원-아트플라워        1       1    0         1        1     1     1   
           학원-에어로빅        23      23    2        23       23    23    23   
           학원-요리          67      67   18        67       67    67    67   
           학원-웅변          30      30    7        30       30    30    30   
           학원-자수/동양자수      3       3    0         3        3     3     3   
           학원-주산암산부기속셈   113     113   25       113      113   113   113   
           학원-지점토          1       1    0         1        1     1     1   
           학원-펜글씨/한문      24      24    1        24       24    24    24   
           합기도장          104     104    1       104      104   104   104   
학원-음악미술무용  서예/서화/미술     1473    1473  228      1473     1473  1473  1473   
           피아노/바이올린/기타  2756    2756  341      2756     2756  2756  2755   
           학원-국악          43      43    2        43       43    43    43   
           학원-무용         214     214   37       214      214   214   214   
학원-자격/국가고시 학원-간호조무사       71      71   11        71       71    71    71   
           학원-건축토목기술       7       7    2         7        7     7     7   
           학원-경찰           4       4    3         4        4     4     4   
           학원-공인중개사       17      17    6        17       17    17    17   
           학원-관광통역         2       2    0         2        2     2     2   
           학원-국가고시         7       7    1         7        7     7     7 

In [9]:
df.columns

Index(['상호명', '상가업소번호', '지점명', '상권업종대분류명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명',
       '시도명', '법정동명', '경도', '위도'],
      dtype='object')

####  '상권업종중분류명' : '학원-컴퓨터'

In [11]:
itaca = df[df['상권업종중분류명']=='학원-컴퓨터']

In [12]:
del df

In [15]:
itaca = itaca[['상호명','지점명','경도','위도']]

In [31]:
import folium
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan',11:'pink'}

df = itaca
cluster2_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng in zip(df.상호명, df.위도, df.경도):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[1],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[1],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster2_map)
cluster2_map
# 지도를 html 파일로 저장하기
# cluster2_map.save('./seoul_mschool_cluster2.html')


In [32]:
import folium
from sklearn import preprocessing
from sklearn import cluster
colors = {-1:'gray', 0:'coral', 1:'blue', 2:'green', 3:'red', 4:'purple', 
          5:'orange', 6:'brown', 7:'brick', 8:'yellow', 9:'magenta', 10:'cyan',11:'pink'}

columns_list =['경도','위도']
X3 = itaca.loc[:, columns_list]
x = preprocessing.StandardScaler().fit(X3).transform(X3)

# dbm = cluster.DBSCAN()#eps=0.2, min_samples=5
dbm = cluster.KMeans(init='k-means++', n_clusters=5, n_init=10)
dbm.fit(x)  

itaca['cluster'] = dbm.labels_   

cluster3_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

for name, lat, lng, clus in zip(itaca.상호명, itaca.위도, itaca.경도, itaca.cluster):  
    folium.CircleMarker([lat, lng],
                        radius=5,                   # 원의 반지름
                        color=colors[clus],         # 원의 둘레 색상
                        fill=True,
                        fill_color=colors[clus],    # 원을 채우는 색
                        fill_opacity=0.7,           # 투명도    
                        popup=name
    ).add_to(cluster3_map)
cluster3_map

C:\Users\epsilon\dev\Anaconda\envs\fortf\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
